## ITXI AI Assessment


In this notebook, we will try to explore OpenAI's APIs and how we can employ them to automate the workflow of an application from getting natural language prompts to executing business logic function behind the scenes.

In this notebook, we will have a model that provides users with visa information depending on their `origin country`, and their `destination country`.


##### Import Modules


In [10]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv

##### Load `.env` Variables


In [11]:
load_dotenv()
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

##### Load Json File


In [12]:
PATH_TO_JSON_FILE = "data/visa_information.json"

try:
    with open(PATH_TO_JSON_FILE) as file:
        visa_information = json.load(file)
except:
    raise Exception("Failed to load json file.")

##### Adjust Json Structure

The current structure is ineffecient for lookups of origin and destination countries, and since we will expect multiple lookups, I decided to switch the levels of the `visa` type and the underlying countries.

From:

`"Origin Country": {
    "visaFree": {
        "Destination Country": {
            "Max Stay": Number
        }
    }
}`

To:

`"Origin Country": {
    "Destination Country" {
        "Visa Type": "visaFree" | "visaOrArrival",
        "Max Stay": Number
    }
}`

For our scenario here (small json, and only simple lookup), this structure will offer more efficient lookups, however, later on if the json file includes all countries in the world, then it will cause a storage overhead, and become a bit unreadable.


In [13]:
new_visa_information = {}

for origin, visa_info in visa_information.items():
    new_visa_information[origin] = {}
    """ 
        {
            origin: {}
        }
    """

    for destination, details in visa_info.get("visaFree", {}).items():
        new_visa_information[origin][destination] = {
            "type": "visaFree",
            "maxStay": details["maxStay"]
        }
    
    for destination, details in visa_info.get("visaOnArrival", {}).items():
        new_visa_information[origin][destination] = {
            "type": "visaOnArrival",
            "maxStay": details["maxStay"]
        }
    
    """
    {
        origin: 
        {
            destination: 
            {
                type: string,
                "maxStay": number    
            }    
        }
    }        
    """

##### Function Description

A `Function Description` is an object that we provide to our model to describe to the model our `Function Calling` function which allows the model to better suggest which function to call and extract the parameters required for the function from natural language.


In [14]:
function_descriptions = [
    {
        "name": "get_visa_information",
        "description": "Get Visa information between two locations",
        "parameters": {
            "type": "object",
            "properties": {
                "origin_country": {
                    "type": "string",
                    "description": "Abbreviation of the country from which the user's passport is issued",
                },
                "destination_country": {
                    "type": "string",
                    "description": "Abbreviation of the country the user is planning on visiting"
                }
            },
            "required": ["origin_country" ,"destination_country"]
        }
    }    
]

##### Business Logic

Here we will create the function that we will execute when the assistant prompts our code to.
When a user asks the assistant for visa information for an origin and destination country, the assistant will parse the natural language and return an object with the function name and its corresponding parameters.


In [68]:
def get_visa_information(data: dict, kwargs: dict):
    origin_country = kwargs.get("origin_country")
    destination_country = kwargs.get("destination_country")
    
    if not origin_country and not destination_country:
        raise ValueError("Both 'origin' and 'destination' countries are required.")
    
    visa_info = data.get(origin_country, {}).get(destination_country, {})
    
    return {
        'origin country': origin_country,
        'destination country': destination_country,
        'visa type': visa_info.get('type', 'visa required'),
        'maxStay': visa_info.get('maxStay', 0)
    }

##### Prompting and Chat Completion

Now we are going to use the user prompts, and the subsequent assistant responses to call the function above and return a proper answer to the user.


In [16]:
user_prompt = "Hello, I'm planning a trip from to Portugal. Can you help me with the visa requirements?"

In [18]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": user_prompt}],
    functions=function_descriptions,
    function_call="auto",
)

In [31]:
assistant_message = completion.choices[0].message.content

print(assistant_message)

Sure, I can help with that! Could you please provide me with the country that issued your passport?


In [20]:
user_prompt_2 = "My passport is from Lebanon."

In [32]:
second_completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=
    [
        {"role": "user", "content": user_prompt}, 
        {"role": "assistant", "content": assistant_message}, 
        {"role": "user", "content": user_prompt_2}
    ],
    functions=function_descriptions,
    function_call="auto",
)

In [71]:
assistant_message_2 = second_completion.choices[0].message.function_call
function_call = eval(assistant_message_2.name)
arguments=json.loads(assistant_message_2.arguments)

print(assistant_message_2)
print(function_call)
print(arguments)

FunctionCall(arguments='{"origin_country":"LB","destination_country":"PT"}', name='get_visa_information')
<function get_visa_information at 0x000001D9B30C6B90>
{'origin_country': 'LB', 'destination_country': 'PT'}


In [72]:
visa_info_result = function_call(new_visa_information, arguments)

print(visa_info_result)

{'origin country': 'LB', 'destination country': 'PT', 'visa type': 'visa required', 'maxStay': 0}


In [81]:
print(second_completion)

ChatCompletion(id='chatcmpl-AtdE5ri1b2F8QZvS8JXXPlod27N6i', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=FunctionCall(arguments='{"origin_country":"LB","destination_country":"PT"}', name='get_visa_information'), tool_calls=None))], created=1737822189, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_72ed7ab54c', usage=CompletionUsage(completion_tokens=23, prompt_tokens=132, total_tokens=155, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [94]:
print(second_completion.choices[0])

Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=FunctionCall(arguments='{"origin_country":"LB","destination_country":"PT"}', name='get_visa_information'), tool_calls=None))


In [101]:
third_completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=
    [
        {"role": "user", "content": user_prompt}, 
        {"role": "assistant", "content": assistant_message}, 
        {"role": "user", "content": user_prompt_2},
        {
            "role": "function",
            "name": assistant_message_2.name, "content": json.dumps(visa_info_result)
        }
    ],
    functions=function_descriptions,
    function_call="none",
)

In [102]:
print(third_completion)

ChatCompletion(id='chatcmpl-Ateq9sMqGujvEAC15uQad9mnrmTAP', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='As a Lebanese passport holder, you will need a visa to enter Portugal. Unfortunately, the maximum stay and specific details regarding the visa are not provided. I recommend checking with the nearest Portuguese embassy or consulate for the most accurate and detailed information regarding visa types, application processes, and any other requirements you may need to fulfill. If you have any other questions or need further assistance, feel free to ask!', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1737828393, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_72ed7ab54c', usage=CompletionUsage(completion_tokens=82, prompt_tokens=169, total_tokens=251, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, a